In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/scripts

/content/drive/MyDrive/Colab Notebooks/scripts


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
import os

In [ ]:
from feature_selection import forwardFeatureSelection

from GenLinCFA import GenLinCFA

from aux_GenLinCFA import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,FS_with_logisticWrapper,compare_methods,compare_methods_class,compute_r2,prepare_target_binary

# GenLinCFA aggregations

## Temp + Prec

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
basins = ['Adda', 'Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_target = "./csv/"
path_features = './features_allvalues/'
destination_folder = './GenLinCFA/temp_prec_moving_avg_4/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train, target_df_val, target_df_test, target_df_trainVal = prepare_target_binary('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
          path=path_target+basin+'.csv', threshold = None, nopeaks = False, window_size = 4)

    eps = 0.37

    actual_path = path_features+basin+'_aggreg.csv'

    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    selected_colnames = FS_with_logisticWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test.csv'

    X_train_wrapper = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_wrapper = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_wrapper = pd.concat([X_train_wrapper, X_validation_wrapper])
    X_test_wrapper = aggregate_test.loc[:,selected_colnames[0:5]]

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train.csv'
    val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val.csv'
    test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test.csv'

    X_train_CMI5 = aggregate_trainVal.loc[:410,selected_colnames[0:5]]
    X_validation_CMI5 = aggregate_trainVal.loc[411:,selected_colnames[0:5]]
    X_train_validation_CMI5 = pd.concat([X_train_CMI5, X_validation_CMI5])
    X_test_CMI5 = aggregate_test.loc[:,selected_colnames[0:5]]

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_genLinCFA_CMI_train.csv'
    val_string = destination_folder + basin + '_genLinCFA_CMI_val.csv'
    test_string = destination_folder + basin + '_genLinCFA_CMI_test.csv'

    X_train_CMI = aggregate_trainVal.loc[:410,selected_colnames]
    X_validation_CMI = aggregate_trainVal.loc[411:,selected_colnames]
    X_train_validation_CMI = pd.concat([X_train_CMI, X_validation_CMI])
    X_test_CMI = aggregate_test.loc[:,selected_colnames]

    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)

    print('###### Binary Classification ######')

    log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

    # CMI
    log_regr.fit(X_train_validation_CMI, target_df_trainVal['mean_std'])
    print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, target_df_trainVal['mean_std']),3))
    print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, target_df_test['mean_std']),3), "\n")

    # CMI best 5
    log_regr.fit(X_train_validation_CMI5, target_df_trainVal['mean_std'])
    print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, target_df_trainVal['mean_std']),3))
    print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, target_df_test['mean_std']),3), "\n")

    # wrapper
    log_regr.fit(X_train_validation_wrapper, target_df_trainVal['mean_std'])
    print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, target_df_trainVal['mean_std']),3))
    print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, target_df_test['mean_std']),3), "\n")


####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -3.170273
1    2001-01-13  0.209996    0.43  2001     2 -1.756328
2    2001-01-21  0.253992    0.38  2001     3 -1.391730
3    2001-01-29  0.271005    0.35  2001     5 -1.250743
4    2001-02-06  0.349900    0.40  2001     6 -0.596941
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.449148    0.40  2009    48  0.225525
407  2009-12-05  0.405264    0.46  2009    49 -0.138143
408  2009-12-13  0.397738    0.53  2009    50 -0.200511
409  2009-12-21  0.397360    0.00  2009    52 -0.203640
410  2009-12-29  0.323312    0.00  2009    53 -0.817278

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
Number of aggregated features: 4

Number of features: 92

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
8
Nu

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che



selected columns: ['cyclostationary_mean_rr_16w_2', 'cyclostationary_mean_rr_16w_1', 'cyclostationary_mean_rr_1w_0', 'cyclostationary_mean_rr_1', 'cyclostationary_mean_tg_16w_0', 'cyclostationary_mean_rr_8', 'cyclostationary_mean_tg_12w_1', 'cyclostationary_mean_rr_3', 'cyclostationary_mean_rr_7', 'cyclostationary_mean_rr_4w_0'], 

validation score: 0.6842105263157895, 

number of selected features: 10

Full model and selected features with wrapper



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.6979655712050078, test score: 0.543859649122807
Aggregate classification train score with FS: 0.6134585289514867, test score: 0.5131578947368421

Full model and best 5 selected features with wrapper



...

Full aggregate classification train score: 0.7464788732394366, test score: 0.6052631578947368
Aggregate classification train score with FS: 0.6588419405320813, test score: 0.7368421052631579

Full model and best 5 selected features with CMI



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7464788732394366, test score: 0.6052631578947368
Aggregate classification train score with FS: 0.6823161189358372, test score: 0.6578947368421053
###### Binary Classification ######
Train accuracy logistic regression CMI:  0.665
Test accuracy logistic regression CMI:  0.754 

Train accuracy logistic regression CMI best 5:  0.673
Test accuracy logistic regression CMI best 5:  0.675 

Train accuracy logistic regression wrapper:  0.624
Test accuracy logistic regression wrapper:  0.706 

####################Ticino####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.264043    0.00  2001     1 -1.367371
1    2001-01-13  0.309331    0.39  2001     2 -0.944476
2    2001-01-21  0.348884    0.47  2001     3 -0.575125
3    2001-01-29  0.346537    0.35  2001     5 -0.597042
4    2001-02-06  0.361559    0.34  2001     6 -0.456760
..          ...       ...     ...   ...   ...       ...
406  2009-1

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(




selected columns: ['cyclostationary_mean_tg_4w_2', 'cyclostationary_mean_rr_16w_0', 'cyclostationary_mean_tg_8w_0', 'cyclostationary_mean_tg_8w_2', 'cyclostationary_mean_tg_24w_0', 'cyclostationary_mean_tg_4w_0', 'cyclostationary_mean_tg_4w_1', 'cyclostationary_mean_rr_4w_0', 'cyclostationary_mean_tg_12w_2', 'cyclostationary_mean_rr_3'], 

validation score: 0.7763157894736842, 

number of selected features: 10

Full model and selected features with wrapper

Full aggregate classification train score: 0.7386541471048513, test score: 0.6008771929824561
Aggregate classification train score with FS: 0.6776212832550861, test score: 0.6666666666666666

Full model and best 5 selected features with wrapper

Full aggregate classification train score: 0.7386541471048513, test score: 0.6008771929824561
Aggregate classification train score with FS: 0.6557120500782473, test score: 0.6710526315789473
----- MI Scores -----
[(24, 0.08050103726906811), (8, 0.07358220470613645), (9, 0.07124600306412965

## Temp + Prec + Snow

In [ ]:
basins = ['Piemonte_Nord','Piemonte_Sud','Adda','Dora','Ticino']
path_target = "./csv/"
path_features_snow = './features_allvalues/snow/'
path_features='./features_allvalues/'

destination_folder = './GenLinCFA/temp_prec_snow/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)
    eps = 0.37
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'

    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    print("End of temp and rain features, start of snow features aggregation \n")

    output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                             'cyclostationary_mean_HS_1w',
                                                                             'cyclostationary_mean_HS_4w',
                                                                             'cyclostationary_mean_HS_8w',
                                                                             'cyclostationary_mean_HS_12w',
                                                                             'cyclostationary_mean_HS_16w',
                                                                             'cyclostationary_mean_HS_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                       min_aggreg = 2, max_aggreg = 3)


    aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal_temp_prec),axis=1)
    aggregate_test = pd.concat((aggregate_test_snow,aggregate_test_temp_prec),axis=1)

    selected_colnames = FS_with_logisticWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train_withSnow.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val_withSnow.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test_withSnow.csv'

    #aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    #aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    #aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test_withSnow.csv'

    #aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    #aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    #aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_genLinCFA_CMI_train_withSnow.csv'
    val_string = destination_folder + basin + '_genLinCFA_CMI_val_withSnow.csv'
    test_string = destination_folder + basin + '_genLinCFA_CMI_test_withSnow.csv'

    #aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    #aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    #aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)


####################Piemonte_Nord####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1 -1.146332
1    2001-01-13  0.494910    0.51  2001     2  0.371173
2    2001-01-21  0.496092    0.51  2001     3  0.379474
3    2001-01-29  0.427992    0.43  2001     5 -0.099118
4    2001-02-06  0.400512    0.41  2001     6 -0.292244
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.363952    0.37  2009    48 -0.549184
407  2009-12-05  0.400487    0.40  2009    49 -0.292423
408  2009-12-13  0.506771    0.52  2009    50  0.454529
409  2009-12-21  0.387530    0.53  2009    52 -0.383480
410  2009-12-29  0.279894    0.27  2009    53 -1.139931

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 89

1
1
1
1
1
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
12
1
1

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7949921752738655, test score: 0.5964912280701754
Aggregate classification train score with FS: 0.6948356807511737, test score: 0.6359649122807017

Full model and best 5 selected features with wrapper



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7949921752738655, test score: 0.5964912280701754
Aggregate classification train score with FS: 0.6713615023474179, test score: 0.6359649122807017
----- MI Scores -----
[(6, 0.22992309740299527), (5, 0.21682534744943052), (0, 0.20648092036053264), (8, 0.16033204451061742), (14, 0.15919013942720192), (7, 0.15900850716874246), (13, 0.13033781622985435), (4, 0.12108049669876571), (15, 0.11847425054410846), (3, 0.11056867351178358), (10, 0.109102741184146), (21, 0.10386697716709256), (11, 0.09911890432433872), (2, 0.08553625800782601), (1, 0.07854238177271229), (16, 0.07737999451036957), (27, 0.07160452677225713), (99, 0.0696552299811048), (12, 0.06642798813699634), (19, 0.06524050330848714), (114, 0.06489417909526018), (22, 0.0647775973720923), (93, 0.06390412024251571), (95, 0.06367830497467163), (9, 0.06208694745187174), (96, 0.06196935676862346), (43, 0.06195971101675305), (102, 0.060353017962879424), (35, 0.060273692408838515), (98, 0.060105

...

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7417840375586855, test score: 0.6140350877192983
Aggregate classification train score with FS: 0.6697965571205008, test score: 0.6535087719298246
----- MI Scores -----
[(0, 0.1584228972240006), (2, 0.1569980537128593), (17, 0.0763234536076634), (22, 0.07212690205916393), (16, 0.07028585194925412), (4, 0.06858895407570223), (19, 0.06643443388585744), (21, 0.06329307871298487), (14, 0.0612092557291343), (18, 0.060858722509433257), (6, 0.06025808849519768), (48, 0.05871789878382834), (15, 0.05815706083531807), (20, 0.056323847793093886), (9, 0.05549602395344354), (53, 0.054899875197957825), (8, 0.05147378600380178), (51, 0.05037022008290984), (49, 0.048958793309502306), (24, 0.04629049926461868), (70, 0.043381055201335235), (38, 0.04319332301250448), (37, 0.04140224111744815), (60, 0.03988064882554653), (59, 0.03928885623448532), (54, 0.03917062658063098), (28, 0.03870989674028667), (50, 0.03821540005602066), (7, 0.036321933095037305), (25, 0.0

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7417840375586855, test score: 0.6140350877192983
Aggregate classification train score with FS: 0.543035993740219, test score: 0.5394736842105263

Full model and best 5 selected features with CMI

Full aggregate classification train score: 0.7417840375586855, test score: 0.6140350877192983
Aggregate classification train score with FS: 0.543035993740219, test score: 0.5394736842105263


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### epsilon value

In [ ]:
basin = "Piemonte_Sud"

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/'
path_features='./features_allvalues/'

print('####################' + basin + '####################')

target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)
eps = 0.37
actual_path = path_features+basin+'_aggreg.csv'
snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'

output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                          'cyclostationary_mean_HS_1w',
                                                                          'cyclostationary_mean_HS_4w',
                                                                          'cyclostationary_mean_HS_8w',
                                                                          'cyclostationary_mean_HS_12w',
                                                                          'cyclostationary_mean_HS_16w',
                                                                          'cyclostationary_mean_HS_24w'
                                                                        ],
                                                                    target_df_trainVal, eps=eps,
                                                                    max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                    min_aggreg = 2, max_aggreg = 3)

####################Piemonte_Sud####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278060    0.09  2001     1 -0.967137
1    2001-01-13  0.445159    0.48  2001     2  0.070382
2    2001-01-21  0.488982    0.52  2001     3  0.342478
3    2001-01-29  0.362487    0.37  2001     5 -0.442927
4    2001-02-06  0.430732    0.45  2001     6 -0.019192
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.430379    0.44  2009    48 -0.021388
407  2009-12-05  0.419919    0.43  2009    49 -0.086330
408  2009-12-13  0.526648    0.55  2009    50  0.576347
409  2009-12-21  0.457440    0.61  2009    52  0.146632
410  2009-12-29  0.301938    0.38  2009    53 -0.818877

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 17

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1

In [ ]:
basin = "Piemonte_Sud"

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/'
path_features='./features_allvalues/'

print('####################' + basin + '####################')

target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)
eps = 0.45
actual_path = path_features+basin+'_aggreg.csv'
snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'

output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                          'cyclostationary_mean_HS_1w',
                                                                          'cyclostationary_mean_HS_4w',
                                                                          'cyclostationary_mean_HS_8w',
                                                                          'cyclostationary_mean_HS_12w',
                                                                          'cyclostationary_mean_HS_16w',
                                                                          'cyclostationary_mean_HS_24w'
                                                                        ],
                                                                    target_df_trainVal, eps=eps,
                                                                    max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                    min_aggreg = 2, max_aggreg = 3)

####################Piemonte_Sud####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278060    0.09  2001     1 -0.967137
1    2001-01-13  0.445159    0.48  2001     2  0.070382
2    2001-01-21  0.488982    0.52  2001     3  0.342478
3    2001-01-29  0.362487    0.37  2001     5 -0.442927
4    2001-02-06  0.430732    0.45  2001     6 -0.019192
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.430379    0.44  2009    48 -0.021388
407  2009-12-05  0.419919    0.43  2009    49 -0.086330
408  2009-12-13  0.526648    0.55  2009    50  0.576347
409  2009-12-21  0.457440    0.61  2009    52  0.146632
410  2009-12-29  0.301938    0.38  2009    53 -0.818877

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 17

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1

In [ ]:
basin = "Piemonte_Sud"

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/'
path_features='./features_allvalues/'

print('####################' + basin + '####################')

target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)
eps = 0.55
actual_path = path_features+basin+'_aggreg.csv'
snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'

output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                          'cyclostationary_mean_HS_1w',
                                                                          'cyclostationary_mean_HS_4w',
                                                                          'cyclostationary_mean_HS_8w',
                                                                          'cyclostationary_mean_HS_12w',
                                                                          'cyclostationary_mean_HS_16w',
                                                                          'cyclostationary_mean_HS_24w'
                                                                        ],
                                                                    target_df_trainVal, eps=eps,
                                                                    max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                    min_aggreg = 2, max_aggreg = 3)

####################Piemonte_Sud####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278060    0.09  2001     1 -0.967137
1    2001-01-13  0.445159    0.48  2001     2  0.070382
2    2001-01-21  0.488982    0.52  2001     3  0.342478
3    2001-01-29  0.362487    0.37  2001     5 -0.442927
4    2001-02-06  0.430732    0.45  2001     6 -0.019192
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.430379    0.44  2009    48 -0.021388
407  2009-12-05  0.419919    0.43  2009    49 -0.086330
408  2009-12-13  0.526648    0.55  2009    50  0.576347
409  2009-12-21  0.457440    0.61  2009    52  0.146632
410  2009-12-29  0.301938    0.38  2009    53 -0.818877

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 17

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1

In [ ]:
basin = "Piemonte_Sud"

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/'
path_features='./features_allvalues/'

print('####################' + basin + '####################')

target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)
eps = 0.75
actual_path = path_features+basin+'_aggreg.csv'
snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'

output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                          'cyclostationary_mean_HS_1w',
                                                                          'cyclostationary_mean_HS_4w',
                                                                          'cyclostationary_mean_HS_8w',
                                                                          'cyclostationary_mean_HS_12w',
                                                                          'cyclostationary_mean_HS_16w',
                                                                          'cyclostationary_mean_HS_24w'
                                                                        ],
                                                                    target_df_trainVal, eps=eps,
                                                                    max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                    min_aggreg = 2, max_aggreg = 3)

####################Piemonte_Sud####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278060    0.09  2001     1 -0.967137
1    2001-01-13  0.445159    0.48  2001     2  0.070382
2    2001-01-21  0.488982    0.52  2001     3  0.342478
3    2001-01-29  0.362487    0.37  2001     5 -0.442927
4    2001-02-06  0.430732    0.45  2001     6 -0.019192
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.430379    0.44  2009    48 -0.021388
407  2009-12-05  0.419919    0.43  2009    49 -0.086330
408  2009-12-13  0.526648    0.55  2009    50  0.576347
409  2009-12-21  0.457440    0.61  2009    52  0.146632
410  2009-12-29  0.301938    0.38  2009    53 -0.818877

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 17

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1

## Temp + Prec + Snow + Lakes

In [ ]:
basins = ['Adda', 'Ticino', 'Oglio_Iseo']

path_target = "./csv/"
path_features_snow = './features_allvalues/snow/'
path_features='./features_allvalues/'
path_features_lakes='./lakes/'

destination_folder = './GenLinCFA/temp_prec_snow_lakes/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)
    eps = 0.37
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_snowDepth_aggreg_allCoord.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(snow_actual_path):
      output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS',
                                                                              'cyclostationary_mean_HS_1w',
                                                                              'cyclostationary_mean_HS_4w',
                                                                              'cyclostationary_mean_HS_8w',
                                                                              'cyclostationary_mean_HS_12w',
                                                                              'cyclostationary_mean_HS_16w',
                                                                              'cyclostationary_mean_HS_24w'
                                                                              ],
                                                                        target_df_trainVal, eps=eps,
                                                                        max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                       min_aggreg = 2, max_aggreg = 3)
      aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test_snow,aggregate_test),axis=1)

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)
      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((aggregate_trainVal, df_lakes_trainVal),axis=1)
      aggregate_test = pd.concat((aggregate_test, df_lakes_test),axis=1)

    # dividere in trainVal e test, aggregare al resto

    selected_colnames = FS_with_logisticWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_genLinCFA_CMI_train_withSnowLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_CMI_val_withSnowLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_CMI_test_withSnowLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

1
1
1
1
1
1
1
1
4
Number of aggregated features: 4

Number of features: 92

1
1
1
1
1
1
1
1
1
1
1
3
Number of aggregated features: 3

Number of features: 92

1
1


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/line



selected columns: ['cyclostationary_mean_tg_1w_1', 'cyclostationary_mean_rr_8w_6', 'cyclostationary_mean_HS_12w_3', 'cyclostationary_mean_tg_12w_5', 'cyclostationary_mean_tg_24w_2', 'cyclostationary_mean_HS_3', 'cyclostationary_mean_tg_24w_5', 'cyclostationary_mean_rr_4w_3', 'cyclostationary_mean_tg_24w_3', 'cyclostat_release_Como_24w', 'cyclostationary_mean_tg_24w_6', 'cyclostat_level_Como_4w', 'cyclostationary_mean_rr_8w_1', 'cyclostationary_mean_rr_12w_2', 'cyclostationary_mean_rr_12w_6', 'cyclostationary_mean_rr_4w_5', 'cyclostat_inflow_Como_12w', 'cyclostationary_mean_rr_8w_2', 'cyclostat_inflow_Como_16w', 'cyclostationary_mean_rr_12w_7', 'cyclostat_inflow_Como', 'cyclostationary_mean_rr_4w_0', 'cyclostationary_mean_rr_12w_4', 'cyclostationary_mean_HS_0', 'cyclostationary_mean_HS_16w_3', 'cyclostationary_mean_rr_1w_0', 'cyclostationary_mean_tg_12w_2', 'cyclostat_release_Como_16w', 'cyclostationary_mean_rr_8w_5', 'cyclostationary_mean_rr_16w_1', 'cyclostationary_mean_tg_1w_2', 'c

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7636932707355243, test score: 0.6228070175438597


...

Full aggregate classification train score: 0.7543035993740219, test score: 0.6359649122807017
Aggregate classification train score with FS: 0.6823161189358372, test score: 0.7192982456140351
----- MI Scores -----
[(59, 0.08549571812014782), (32, 0.06853188279074499), (63, 0.06762208117273773), (62, 0.06685091011617787), (2, 0.06586032186024908), (74, 0.06479059467693574), (34, 0.06470526276221414), (4, 0.06464291050732981), (0, 0.06444463931047233), (76, 0.06292254806123161), (1, 0.061927233560351916), (69, 0.057835986691592005), (5, 0.05555752149823557), (31, 0.054733806044338924), (68, 0.05235162638183522), (37, 0.0502229974523168), (30, 0.049704508476126535), (71, 0.04935697629717276), (60, 0.04881175347541353), (28, 0.04780120789141581), (6, 0.047640393563938004), (66, 0.04725190813865622), (35, 0.047201117888292746), (77, 0.04645264817062005), (40, 0.045367917696951164), (75, 0.04507134060200013), (70, 0.04287944667180395), (7, 0.04282520406291697), (39, 0.04220033667123514), (78,

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7543035993740219, test score: 0.6359649122807017
Aggregate classification train score with FS: 0.6353677621283255, test score: 0.6140350877192983

Full model and best 5 selected features with CMI



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Full aggregate classification train score: 0.7543035993740219, test score: 0.6359649122807017
Aggregate classification train score with FS: 0.6306729264475743, test score: 0.5877192982456141


## Temp + Prec + Lakes

In [ ]:
basins = ['Adda', 'Ticino', 'Oglio_Iseo']
path_target = "./csv/"
path_features='./features_allvalues/'
path_features_lakes='./lakes/'

destination_folder = './GenLinCFA/temp_prec_lakes/'


for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)
    eps = 0.37
    actual_path = path_features+basin+'_aggreg.csv'
    lakes_actual_path = path_features_lakes + 'lakes_' + basin + '_with_aggreg.csv'

    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg',
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w',
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w',
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr',
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w',
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w',
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')

    if os.path.isfile(lakes_actual_path):
      df_lakes = pd.read_csv(lakes_actual_path)

      df_lakes = df_lakes.drop("Unnamed: 0", axis='columns')

      df_lakes_trainVal = df_lakes.loc[(df_lakes['date'] > '2001-01-01') & (df_lakes['date'] <= '2015-01-01'),:]
      df_lakes_trainVal.reset_index(inplace = True, drop = True)
      df_lakes_trainVal = df_lakes_trainVal.iloc[:,3:]
      df_lakes_test = df_lakes.loc[(df_lakes['date'] > '2015-01-01') & (df_lakes['date'] <= '2020-01-01'),:]
      df_lakes_test.reset_index(inplace = True, drop = True)
      df_lakes_test = df_lakes_test.iloc[:,3:]

      aggregate_trainVal = pd.concat((df_lakes_trainVal,aggregate_trainVal_temp_prec),axis=1)
      aggregate_test = pd.concat((df_lakes_test,aggregate_test_temp_prec),axis=1)

    selected_colnames = FS_with_logisticWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_wrapper_best5_train_withLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_wrapper_best5_val_withLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_wrapper_best5_test_withLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    res = {
            "delta" : [],
            "numSelected" : [],
            "selectedFeatures" : []
        }

    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)

    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

    print('\nFull model and selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods_class(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])

    train_string = destination_folder + basin + '_genLinCFA_best5_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_best5_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_best5_CMI_test_withLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = destination_folder + basin + '_genLinCFA_CMI_train_withLakes.csv'
    val_string = destination_folder + basin + '_genLinCFA_CMI_val_withLakes.csv'
    test_string = destination_folder + basin + '_genLinCFA_CMI_test_withLakes.csv'

    aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)

# GenLinCFA results in binary classification

## Target and features

In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
### targets
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_targets = "./csv/"
targets_df_train = pd.DataFrame()
targets_df_val = pd.DataFrame()
targets_df_test = pd.DataFrame()
targets_df_trainVal = pd.DataFrame()

for basin in basins:
    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01',
                                                                                     path=path_targets+basin+'.csv')
    targets_df_train[basin] = target_df_train.apply(lambda x: np.sign(x.mean_std), axis=1)
    targets_df_val[basin] = target_df_val.apply(lambda x: np.sign(x.mean_std), axis=1)
    targets_df_test[basin] = target_df_test.apply(lambda x: np.sign(x.mean_std), axis=1)
    targets_df_trainVal[basin] = target_df_trainVal.apply(lambda x: np.sign(x.mean_std), axis=1)

target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.010645    0.00  2001     1 -2.129508
1    2001-01-13  0.206769    0.00  2001     2 -0.927136
2    2001-01-21  0.267313    0.00  2001     3 -

In [ ]:
def get_features (path_features, basin, train_name, val_name, test_name):
  features_train = pd.read_csv(path_features + basin + train_name)
  features_valid = pd.read_csv(path_features + basin + val_name)
  features_test = pd.read_csv(path_features + basin + test_name)

  features_train_validation = pd.concat([features_train, features_valid])
  return features_train.values, features_valid.values, features_train_validation.values, features_test.values

In [ ]:
basin = 'Adda'

In [ ]:
# Temp Prec
path_features = './GenLinCFA/temp_prec/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_genLinCFA_CMI_train.csv',
                                                                                 '_genLinCFA_CMI_val.csv', '_genLinCFA_CMI_test.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_genLinCFA_best5_CMI_train.csv',
                                                                                     '_genLinCFA_best5_CMI_val.csv', '_genLinCFA_best5_CMI_test.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_genLinCFA_wrapper_best5_train.csv',
                                                                                                 '_genLinCFA_wrapper_best5_val.csv', '_genLinCFA_wrapper_best5_test.csv')

In [ ]:
print ("Features chosen by CMI for " + basin + " \ntaken " + "\033[1m" + "temp, prec"+ "\033[0m" + ":")
path_features = './GenLinCFA/temp_prec/'
features_df = pd.read_csv(path_features + basin + '_genLinCFA_best5_CMI_train.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI for Adda 
taken temp, prec:
0: cyclostationary_mean_rr_0
1: cyclostationary_mean_tg_1w_1
2: cyclostationary_mean_tg_4w_0
3: cyclostationary_mean_rr_1w_1
4: cyclostationary_mean_rr_4w_7


In [ ]:
# Temp Prec Snow
path_features = './GenLinCFA/temp_prec_snow/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_genLinCFA_CMI_train_withSnow.csv',
                                                                                 '_genLinCFA_CMI_val_withSnow.csv', '_genLinCFA_CMI_test_withSnow.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_genLinCFA_best5_CMI_train_withSnow.csv',
                                                                                     '_genLinCFA_best5_CMI_val_withSnow.csv', '_genLinCFA_best5_CMI_test_withSnow.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_genLinCFA_wrapper_best5_train_withSnow.csv',
                                                                                                 '_genLinCFA_wrapper_best5_val_withSnow.csv', '_genLinCFA_wrapper_best5_test_withSnow.csv')

In [ ]:
print ("Features chosen by CMI for " + basin + " taken temp, prec, snow:")
path_features = './GenLinCFA/temp_prec_snow/'
features_df = pd.read_csv(path_features + basin + '_genLinCFA_best5_CMI_val_withSnow.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI for Ticino taken temp, prec, snow:
0: cyclostationary_mean_tg_0
1: cyclostationary_mean_HS_0
2: cyclostationary_mean_rr_1w_0
3: cyclostationary_mean_HS_1w_0


In [ ]:
print ("Features chosen by wrapper for " + basin + " taken temp, prec, snow:")
path_features = './GenLinCFA/temp_prec_snow/'
features_df = pd.read_csv(path_features + basin + '_genLinCFA_wrapper_best5_val_withSnow.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by wrapper for Adda taken temp, prec, snow:
0: cyclostationary_mean_tg_1w_1
1: cyclostationary_mean_rr_8w_6
2: cyclostationary_mean_HS_12w_3
3: cyclostationary_mean_tg_12w_5
4: cyclostationary_mean_tg_24w_2


In [ ]:
# Temp Prec Snow Lakes
path_features = './GenLinCFA/temp_prec_snow_lakes/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_genLinCFA_CMI_train_withSnowLakes.csv',
                                                                                 '_genLinCFA_CMI_val_withSnowLakes.csv', '_genLinCFA_CMI_test_withSnowLakes.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_genLinCFA_best5_CMI_train_withSnowLakes.csv',
                                                                                     '_genLinCFA_best5_CMI_val_withSnowLakes.csv', '_genLinCFA_best5_CMI_test_withSnowLakes.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_genLinCFA_wrapper_best5_train_withSnowLakes.csv',
                                                                                                 '_genLinCFA_wrapper_best5_val_withSnowLakes.csv', '_genLinCFA_wrapper_best5_test_withSnowLakes.csv')

In [ ]:
print ("Features chosen by wrapper for " + basin + " \n taken " + "\033[1m" + "temp, prec, lakes, snow"+ "\033[0m" + ":")
path_features = './GenLinCFA/temp_prec_snow_lakes/'
features_df = pd.read_csv(path_features + basin + '_genLinCFA_wrapper_best5_train_withSnowLakes.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by wrapper for Ticino 
 taken temp, prec, lakes, snow:
0: cyclostationary_mean_tg_1w_1
1: cyclostationary_mean_HS_4w_1
2: cyclostat_level_Lugano_12w
3: cyclostationary_mean_HS_12w_0
4: cyclostationary_mean_tg_8w_1


In [ ]:
print ("Features chosen by CMI for " + basin + " \n taken " + "\033[1m" + "temp, prec, lakes, snow"+ "\033[0m" + ":")
path_features = './GenLinCFA/temp_prec_snow_lakes/'
features_df = pd.read_csv(path_features + basin + '_genLinCFA_best5_CMI_train_withSnowLakes.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI for Ticino 
 taken temp, prec, lakes, snow:
0: cyclostationary_mean_HS_0
1: cyclostationary_mean_HS_1w_0


In [ ]:
# Temp Prec Lakes
path_features = './GenLinCFA/temp_prec_lakes/'

# CMI
X_train_CMI, X_validation_CMI, X_train_validation_CMI, X_test_CMI = get_features(path_features, basin, '_genLinCFA_CMI_train_withLakes.csv',
                                                                                 '_genLinCFA_CMI_val_withLakes.csv', '_genLinCFA_CMI_test_withLakes.csv')

# CMI_best_5
X_train_CMI5, X_validation_CMI5, X_train_validation_CMI5, X_test_CMI5 = get_features(path_features, basin, '_genLinCFA_best5_CMI_train_withLakes.csv',
                                                                                     '_genLinCFA_best5_CMI_val_withLakes.csv', '_genLinCFA_best5_CMI_test_withLakes.csv')

# wrapper
X_train_wrapper, X_validation_wrapper, X_train_validation_wrapper, X_test_wrapper = get_features(path_features, basin, '_genLinCFA_wrapper_best5_train_withLakes.csv',
                                                                                                 '_genLinCFA_wrapper_best5_val_withLakes.csv', '_genLinCFA_wrapper_best5_test_withLakes.csv')

In [ ]:
print ("Features chosen by wrapper for " + basin + " \ntaken "+ "\033[1m" + "temp, prec, lakes:" + "\033[0m")
path_features = './GenLinCFA/temp_prec_lakes/'
features_df = pd.read_csv(path_features + basin + '_genLinCFA_wrapper_best5_train_withLakes.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by wrapper for Oglio_Iseo 
taken temp, prec, lakes:
0: cyclostationary_mean_tg_2
1: cyclostationary_mean_rr_4w_0
2: cyclostat_release_Iseo_24w
3: cyclostationary_mean_rr_24w_2
4: cyclostationary_mean_rr_3


In [ ]:
print ("Features chosen by CMI for " + basin + " \ntaken "+ "\033[1m" + "temp, prec, lakes:" + "\033[0m")
features_df = pd.read_csv(path_features + basin + '_genLinCFA_best5_CMI_train_withLakes.csv')
idx_dic = {}
for col in features_df.columns:
    idx_dic[col] = features_df.columns.get_loc(col)
print('\n'.join(f'{value}: {key}' for key, value in idx_dic.items()))

Features chosen by CMI for Oglio_Iseo 
taken temp, prec, lakes:
0: cyclostat_level_Iseo
1: cyclostationary_mean_rr_4w_3
2: cyclostationary_mean_rr_24w_0
3: cyclostationary_mean_tg_8w_2
4: cyclostationary_mean_rr_16w_1


## Single Task

In [ ]:
log_regr = LogisticRegression(solver='lbfgs', random_state = 42)

# CMI
log_regr.fit(X_train_validation_CMI, targets_df_trainVal[basin])
print("Train accuracy logistic regression CMI: ", round(log_regr.score(X_train_validation_CMI, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression CMI: ", round(log_regr.score(X_test_CMI, targets_df_test[basin]),3), "\n")

# CMI best 5
log_regr.fit(X_train_validation_CMI5, targets_df_trainVal[basin])
print("Train accuracy logistic regression CMI best 5: ", round(log_regr.score(X_train_validation_CMI5, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression CMI best 5: ", round(log_regr.score(X_test_CMI5, targets_df_test[basin]),3), "\n")

# wrapper
log_regr.fit(X_train_validation_wrapper, targets_df_trainVal[basin])
print("Train accuracy logistic regression wrapper: ", round(log_regr.score(X_train_validation_wrapper, targets_df_trainVal[basin]),3))
print("Test accuracy logistic regression wrapper: ", round(log_regr.score(X_test_wrapper, targets_df_test[basin]),3), "\n")

Train accuracy logistic regression CMI:  0.565
Test accuracy logistic regression CMI:  0.474 

Train accuracy logistic regression CMI best 5:  0.565
Test accuracy logistic regression CMI best 5:  0.474 

Train accuracy logistic regression wrapper:  0.646
Test accuracy logistic regression wrapper:  0.671 



## Multi Task

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def MTL_scores(clust_basins, df_train, df_val, df_test, targets_df_train, targets_df_val, targets_df_test):

    colnames = [x for x in df_train.columns if x.startswith(tuple(clust_basins))]

    clusterdf_train_withClass = pd.DataFrame()
    clusterdf_val_withClass = pd.DataFrame()
    clusterdf_test_withClass = pd.DataFrame()

    for i in range(len(clust_basins)):
        clusterdf_train_withClass = pd.concat((clusterdf_train_withClass,pd.concat((df_train[colnames],pd.DataFrame(1+i*np.ones(len(df_train)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_val_withClass = pd.concat((clusterdf_val_withClass,pd.concat((df_val[colnames],pd.DataFrame(1+i*np.ones(len(df_val)),columns=['basin'])),axis=1)),axis=0)
        clusterdf_test_withClass = pd.concat((clusterdf_test_withClass,pd.concat((df_test[colnames],pd.DataFrame(1+i*np.ones(len(df_test)),columns=['basin'])),axis=1)),axis=0)

    for i in range(len(clust_basins)):
        clusterdf_train_withClass[clust_basins[i]] = clusterdf_train_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_val_withClass[clust_basins[i]] = clusterdf_val_withClass.apply(lambda x: int(x.basin==i+1),axis=1)
        clusterdf_test_withClass[clust_basins[i]] = clusterdf_test_withClass.apply(lambda x: int(x.basin==i+1),axis=1)

    clusterdf_train_withClass = clusterdf_train_withClass.loc[:,clusterdf_train_withClass.columns != 'basin']
    clusterdf_val_withClass = clusterdf_val_withClass.loc[:,clusterdf_val_withClass.columns != 'basin']
    clusterdf_test_withClass = clusterdf_test_withClass.loc[:,clusterdf_test_withClass.columns != 'basin']

    targets_df_train_unfolded = pd.DataFrame()
    targets_df_val_unfolded = pd.DataFrame()
    targets_df_test_unfolded = pd.DataFrame()

    for basin in clust_basins:
        targets_df_train_unfolded =  pd.concat((targets_df_train_unfolded,targets_df_train[basin]),axis=0)
        targets_df_val_unfolded =  pd.concat((targets_df_val_unfolded,targets_df_val[basin]),axis=0)
        targets_df_test_unfolded =  pd.concat((targets_df_test_unfolded,targets_df_test[basin]),axis=0)
    targets_df_train_unfolded = targets_df_train_unfolded.reset_index(drop=True)
    targets_df_val_unfolded = targets_df_val_unfolded.reset_index(drop=True)
    targets_df_test_unfolded = targets_df_test_unfolded.reset_index(drop=True)

    # same scores changing the solver, some differences changing penalty, some improve with l1
    model_ohe = LogisticRegression(max_iter = 500)
    model_ohe.fit(pd.concat((clusterdf_train_withClass,clusterdf_val_withClass)).values,pd.concat((targets_df_train_unfolded,targets_df_val_unfolded)).values.ravel())

    for basin in clust_basins:
        print(basin)
        res = model_ohe.predict(clusterdf_test_withClass.loc[clusterdf_test_withClass[basin]==1].values)
        print(accuracy_score(targets_df_test[basin].values.ravel(), res))

### Features

In [ ]:
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

#### Temp Prec

In [ ]:
### wrapper best 5 features
path_features = './GenLinCFA/temp_prec/'

best5_wrapper_fulldf_train = pd.DataFrame()
best5_wrapper_fulldf_val = pd.DataFrame()
best5_wrapper_fulldf_test = pd.DataFrame()

for basin in basins:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './GenLinCFA/temp_prec/'

CMI_fulldf_train = pd.DataFrame()
CMI_fulldf_val = pd.DataFrame()
CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './GenLinCFA/temp_prec/'

best5_CMI_fulldf_train = pd.DataFrame()
best5_CMI_fulldf_val = pd.DataFrame()
best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Snow

In [ ]:
### wrapper best 5 features
path_features = './GenLinCFA/temp_prec_snow/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()
basins_snow = ['Piemonte_Nord', 'Piemonte_Sud', 'Ticino', 'Dora', 'Adda']

for basin in basins_snow:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train_withSnow.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val_withSnow.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test_withSnow.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './GenLinCFA/temp_prec_snow/'

#CMI_fulldf_train = pd.DataFrame() commentate per mantenere i dati dei bacini senza neve
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_snow:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test_withSnow.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './GenLinCFA/temp_prec_snow/'

#best5_CMI_fulldf_train = pd.DataFrame()
#best5_CMI_fulldf_val = pd.DataFrame()
#best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_snow:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train_withSnow.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val_withSnow.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test_withSnow.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Snow Lakes

In [ ]:
### wrapper best 5 features
path_features = './GenLinCFA/temp_prec_snow_lakes/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()
basins_lakes = ['Adda', 'Oglio_Iseo', 'Ticino']

for basin in basins_lakes:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train_withSnowLakes.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val_withSnowLakes.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test_withSnowLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './GenLinCFA/temp_prec_snow_lakes/'

#CMI_fulldf_train = pd.DataFrame()
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test_withSnowLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './GenLinCFA/temp_prec_snow_lakes/'

#best5_CMI_fulldf_train = pd.DataFrame()
#best5_CMI_fulldf_val = pd.DataFrame()
#best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train_withSnowLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val_withSnowLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test_withSnowLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

#### Temp Prec Lakes

In [ ]:
### wrapper best 5 features
path_features = './GenLinCFA/temp_prec_lakes/'

#best5_wrapper_fulldf_train = pd.DataFrame()
#best5_wrapper_fulldf_val = pd.DataFrame()
#best5_wrapper_fulldf_test = pd.DataFrame()

basins_lakes = ['Adda', 'Oglio_Iseo', 'Ticino']

for basin in basins_lakes:
    best5_wrapper_train_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_train_withLakes.csv')
    best5_wrapper_val_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_val_withLakes.csv')
    best5_wrapper_test_temp = pd.read_csv(path_features+basin+'_genLinCFA_wrapper_best5_test_withLakes.csv')
    best5_wrapper_fulldf_train[basin+'_'+best5_wrapper_train_temp.columns.values] = best5_wrapper_train_temp
    best5_wrapper_fulldf_val[basin+'_'+best5_wrapper_val_temp.columns.values] = best5_wrapper_val_temp
    best5_wrapper_fulldf_test[basin+'_'+best5_wrapper_test_temp.columns.values] = best5_wrapper_test_temp

In [ ]:
### CMI features
path_features = './GenLinCFA/temp_prec_lakes/'

#CMI_fulldf_train = pd.DataFrame()
#CMI_fulldf_val = pd.DataFrame()
#CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_CMI_test_withLakes.csv')
    CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

In [ ]:
### CMI best5 features
path_features = './GenLinCFA/temp_prec_lakes/'

#best5_CMI_fulldf_train = pd.DataFrame()
#best5_CMI_fulldf_val = pd.DataFrame()
#best5_CMI_fulldf_test = pd.DataFrame()

for basin in basins_lakes:
    train_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_train_withLakes.csv')
    val_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_val_withLakes.csv')
    test_temp = pd.read_csv(path_features+basin+'_genLinCFA_best5_CMI_test_withLakes.csv')
    best5_CMI_fulldf_train[basin+'_'+train_temp.columns.values] = train_temp
    best5_CMI_fulldf_val[basin+'_'+val_temp.columns.values] = val_temp
    best5_CMI_fulldf_test[basin+'_'+test_temp.columns.values] = test_temp

### Wrapper

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.7017543859649122
Emiliani2
0.7807017543859649
Garda_Mincio
0.7807017543859649


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6008771929824561
Lambro_Olona
0.6447368421052632
Oglio_Iseo
0.6140350877192983
Ticino
0.6447368421052632


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.5614035087719298
Piemonte_Nord
0.6447368421052632
Piemonte_Sud
0.6798245614035088


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_wrapper_fulldf_train, df_val=best5_wrapper_fulldf_val, df_test=best5_wrapper_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.6578947368421053
Piemonte_Sud
0.6885964912280702


### CMI

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.6754385964912281
Emiliani2
0.6885964912280702
Garda_Mincio
0.7412280701754386


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.618421052631579
Lambro_Olona
0.6271929824561403
Oglio_Iseo
0.6228070175438597
Ticino
0.6271929824561403


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.5263157894736842
Piemonte_Nord
0.6008771929824561
Piemonte_Sud
0.6008771929824561


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=CMI_fulldf_train, df_val=CMI_fulldf_val, df_test=CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.5789473684210527
Piemonte_Sud
0.5833333333333334


### CMI best 5

In [ ]:
MTL_scores(clust_basins=['Emiliani1', 'Emiliani2', 'Garda_Mincio'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Emiliani1
0.6754385964912281
Emiliani2
0.706140350877193
Garda_Mincio
0.7236842105263158


In [ ]:
MTL_scores(clust_basins=['Adda', 'Lambro_Olona', 'Oglio_Iseo', 'Ticino'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Adda
0.6798245614035088
Lambro_Olona
0.7017543859649122
Oglio_Iseo
0.6885964912280702
Ticino
0.6973684210526315


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
MTL_scores(clust_basins=['Dora', 'Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Dora
0.5789473684210527
Piemonte_Nord
0.6403508771929824
Piemonte_Sud
0.6271929824561403


In [ ]:
MTL_scores(clust_basins=['Piemonte_Nord', 'Piemonte_Sud'], df_train=best5_CMI_fulldf_train, df_val=best5_CMI_fulldf_val, df_test=best5_CMI_fulldf_test, targets_df_train=targets_df_train, targets_df_val=targets_df_val, targets_df_test=targets_df_test)

Piemonte_Nord
0.618421052631579
Piemonte_Sud
0.5789473684210527
